Example notebook for pulling in one year of IFS data and combining with madis

In [ ]:
import os
import xarray as xr
import glob
import numpy as np
import sys
import importlib
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
sys.path.insert(0, "/home/kylabazlen/")
processing = importlib.import_module("2026_nvhackathon.validation.gitprocessing")
subset_grid_to_point_xy = processing.subset_grid_to_point_xy
forecast_obs_merge = processing.forecast_obs_merge
convert_1d_to_2d_latlon = processing.convert_1d_to_2d_latlon
metrics = importlib.import_module("2026_nvhackathon.validation.metrics")
add_gof_stats = metrics.add_gof_stats

In [ ]:
observations = '/project/cowy-nvhackathon/cowy-wildfire/data/observations/cowy_madis_metar_mesonet_2024.nc'
ifs_paths = sorted(glob.glob("/project/cowy-nvhackathon/cowy-wildfire/data/nwp/ifs_yearly/*"))

In [ ]:
# Open the observations file
obs_ds = xr.open_dataset(observations) #space, time diminsions

# Open the first IFS file
ifs_f72 = xr.open_dataset(ifs_paths[0]) #time, latitude, longitude diminsions

In [ ]:
#drop stations with all NA windspeed data
has_data_mask = ~obs_ds['windspeed_10m'].isnull().all(dim='time')

obs_ds_clean = obs_ds.sel(space=has_data_mask)

print(f"Original stations: {obs_ds.dims['space']}")
print(f"Stations with data: {obs_ds_clean.dims['space']}")

In [ ]:
ds = convert_1d_to_2d_latlon(ds = ifs_f72)
ifs_subset = subset_grid_to_point_xy(ds=ds, point_ds=obs_ds_clean)
all_data = forecast_obs_merge(ds1=ifs_subset, ds2=obs_ds_clean, ds1_timevar="valid_time", ds2_timevar="time")

In [ ]:
#add windspeed error as a variable
all_data['ws_error'] = all_data['ws_10'] - all_data['obs_windspeed_10m']

In [ ]:
all_data_gof = add_gof_stats(ds = all_data, var1 = "ws_10", var2 = "obs_windspeed_10m")